# Setup

In [1]:
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.5f}'.format}, suppress = True)

from scipy.interpolate import interpn
from scipy.interpolate import RegularGridInterpolator

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib notebook

MEASFILENAME      = "measurements.txt"
MAGFIELDFILENAMES = [["r25s5", "r25s2_5", "r25s1_25"], ["r50s10", "r50s5", "r50s2_5"], ["r100s20", "r100s10", "r100s5"]]

for i in range(len(MAGFIELDFILENAMES)):
    for j in range(len(MAGFIELDFILENAMES[i])):
        MAGFIELDFILENAMES[i][j] = "magfields/" + MAGFIELDFILENAMES[i][j] + ".txt"

In [2]:
# For some reason this commands needs to be read twice to actually work
%matplotlib notebook

# File reading

In [3]:
# Reads a measurements file and returns an array of measurement locations and measured data
def read_meas_file(filename):
    Z = []
    m = []
    f = open(filename, "r")
    for line in f:
        meas = line.replace("\n", "").split(' ')
        for i in range(len(meas)):
            meas[i] = float(meas[i])
        Z.append(meas[0])
        m.append([meas[0], meas[1], meas[2], meas[3], meas[4], meas[5]])
    f.close()
    Z = np.array(Z)
    for i in range(len(m)):
        m[i] = np.array(m[i])
    return (Z, m)

# Reads a BField file and returns a dictionary containing a set of vectors representing the field
def read_magfield_file(filename):
    B = {}
    f = open(filename, "r")
    for line in f:
        mag_meas = line.replace("\n", "").split(' ')
        for i in range(len(mag_meas)):
            mag_meas[i] = float(mag_meas[i])
        B[(mag_meas[0], mag_meas[1], mag_meas[2])] = (mag_meas[3], mag_meas[4], mag_meas[5])
    f.close()
    return B

# BField functions

In [28]:
# Gets arrays of x, y, z, bx, by and bz in a specific plane on z0
def get_b(B, z0):
    b = ([], [], [], [], [], [])
    for key in B:
        if key[2] <= z0:
            b[0].append(key[0])    # x
            b[1].append(key[1])    # y
            b[2].append(key[2])    # z
            b[3].append(B[key][0]) # bx
            b[4].append(B[key][1]) # by
            b[5].append(B[key][2]) # bz
    return b

# Creates and returns an interpolator for the magnetic field (bx,by,bz) at a given z plane (z0)
def get_z0_interpolator(B, z0):
    (x, y, z, bx, by, bz) = get_b(B, z0)
    
    # pre-process x and y
    grid_x = list(set(x))
    grid_y = list(set(y))
    grid_x.sort()
    grid_y.sort()
    grid_x = np.array(grid_x)
    grid_y = np.array(grid_y)
    
    print(x)
    print(grid_x)
    
    # pre-process bx, by and bz
    grid_bx = []
    grid_by = []
    grid_bz = []

    for i in range(len(grid_x)):
        grid_bx.append([])
        grid_by.append([])
        grid_bz.append([])
        for j in range(len(grid_y)):
            grid_bx[i].append(bx[i*len(grid_x)+j])
            grid_by[i].append(by[i*len(grid_x)+j])
            grid_bz[i].append(bz[i*len(grid_x)+j])
    grid_bx = np.array(grid_bx)
    grid_by = np.array(grid_by)
    grid_bz = np.array(grid_bz)
    
    # obtain and return the interpolator
    bx_est = RegularGridInterpolator((grid_x, grid_y), grid_bx, method="linear", bounds_error=True)
    by_est = RegularGridInterpolator((grid_x, grid_y), grid_by, method="linear", bounds_error=True)
    bz_est = RegularGridInterpolator((grid_x, grid_y), grid_bz, method="linear", bounds_error=True)
    
    return (bx_est, by_est, bz_est)

# Plots a sample of the magnetic field at a cross section z0
def plot_b(B, z0):
    (x, y, z, bx, by, bz) = get_b(B, z0)

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.quiver(x, y, z, bx, by, bz, length=3, normalize=True)

    plt.show()

# Transport

In [29]:
# default step size
step_size = 0.2

def get_A(tx, ty, bx, by, bz):
    c = np.sqrt(1 + tx*tx + ty*ty)
    return [c * ( ty * (tx*bx + bz) - (1 + tx*tx) * by), c * (-tx * (ty*by + bz) + (1 + ty*ty) * bx)]

def get_dA(tx, ty, bx, by, bz):
    c2 = 1 + tx*tx + ty*ty
    c  = np.sqrt(c2)
    Ax = c * ( ty * (tx*bx + bz) - (1 + tx*tx) * by)
    Ay = c * (-tx * (ty*by + bz) + (1 + ty*ty) * bx)
    
    dA = []
    dA.append(tx*Ax/c2 + c * ( ty*bx - 2*tx*by)) # dAx/dtx
    dA.append(ty*Ax/c2 + c * ( tx*bx + bz))      # dAx/dty
    dA.append(tx*Ay/c2 + c * (-ty*by - bz))      # dAy/dtx
    dA.append(ty*Ay/c2 + c * (-tx*by + 2*ty*bx)) # dAy/dty
    
    return dA

def k_transport(i, f, ivec, icov, v, Z, B, prnt): # returns fvec and fcov
    # temporary state vector to improve legibility
    z  = ivec[0]
    x  = ivec[1]
    y  = ivec[2]
    tx = ivec[3]
    ty = ivec[4]
    Q  = ivec[5]

    fcov = np.copy(icov)

    # step size computation
    n_steps = (int) (abs((Z[i] - Z[f]) / step_size) + 1)
    s = np.sign(Z[f] - Z[i]) * step_size
    
    for j in range(n_steps):
        if j == n_steps-1:
            s = np.sign(Z[f] - Z[i]) * abs(z - Z[f])
        
        (bx_est, by_est, bz_est) = get_z0_interpolator(B, z)
        print(x)
        print(y)
        (bx, by, bz) = (bx_est([x,y]), by_est([x,y]), bz_est([x,y]))
        
        A  = get_A(tx, ty, bx, by, bz)
        dA = get_dA(tx, ty, bx, by, bz)
        
        # propagate the covariance matrix
        dx_dtx  = s
        dy_dtx  = 0.5*Q*v * s*s * dA[2]
        dty_dtx = Q*v * s * dA[2]
        
        dx_dty  = 0.5*Q*v * s*s * dA[1]
        dy_dty  = s;
        dtx_dty = Q*v * s * dA[1]
        
        dx_dQ   = 0.5*v * s*s * A[0]
        dtx_dQ  = v * s * A[0]
        dy_dQ   = 0.5*v * s*s * A[1]
        dty_dQ  = v * s * A[1];
        
        # Jacobian matrix
        F = np.array(
            [[1, 0,  dx_dtx,  dx_dty,  dx_dQ],
             [0, 1,  dy_dtx,  dy_dty,  dy_dQ],
             [0, 0,       1, dtx_dty, dtx_dQ],
             [0, 0, dty_dtx,       1, dty_dQ],
             [0, 0,       0,       0,      1]]
        )
        
        # C = FCF^T
        fcov = np.dot(np.dot(F, fcov), F.transpose())
        
        # Q process noise matrix estimate
        p = abs(1./Q)
        pz = p / np.sqrt(1 + tx*tx + ty*ty)
        px = tx * pz
        py = ty * pz
        
        t_ov_X0 = np.sign(Z[f] - Z[i]) * s / 14 # ArgonRadLen = 14
        mass = 0.000510998
        if Q > 0:
            mass = 0.938272029
        beta = p*p / np.sqrt(p*p + mass*mass)
        cos_angle = abs((x*px + y*py + z*pz) / np.sqrt(x*x + y*y + z*z) * p)
        path_len = t_ov_X0 / cos_angle
        
        sct_RMS = (0.0136 / (beta)) * np.sqrt(path_len) * (1 + 0.038*np.log(path_len)) # Highland-Lynch-Dahl formula
        
        cov_txtx = (1 + tx*tx) * (1 + tx*tx + ty*ty) * sct_RMS*sct_RMS
        cov_txty = (1 + ty*ty) * (1 + tx*tx + ty*ty) * sct_RMS*sct_RMS
        cov_tyty = tx*ty * (1 + tx*tx + ty*ty) * sct_RMS*sct_RMS
        
        if s > 0:
            fcov[2,2] += cov_txtx
            fcov[2,3] += cov_txty
            fcov[3,2] += cov_txty
            fcov[3,3] += cov_tyty
        
        # propagate the state vector
        z += s
        x += tx * s + 0.5 * Q*v * A[0] * s*s
        y += ty * s + 0.5 * Q*v * A[1] * s*s
        tx += Q * v * A[0] * s
        ty += Q * v * A[1] * s
        
    fvec = np.array([z, x, y, tx, ty, Q])
    if prnt >= 2:
        print("  transport:  ", fvec)
    if prnt >= 3:
        print("    matrix %2d:\n" % f, fcov)
    return (fvec, fcov)

# Filter

In [30]:
def getH(y, s, w, l):
    h1 = -np.tan(np.radians(6*s))
    return [1, h1 - w * (4/l) * (1 - y/(l/2))]

def geth(x, y, s, w, l):
    return x - np.tan(np.radians(6*s)) * y + w*(1 - y/(l/2)) * (1 - y/(l/2))

def k_filter(k, ivec, icov, chi2, prnt): # returns fvec, fcov and chi2
    # filter the covariance matrix
    if abs(np.linalg.det(icov)) < 1.e-30:
        print("error, initial covariance matrix is singular!")
        return
    
    K = np.zeros(5)
    V = abs(m[k][2])
    H = getH(ivec[2], m[k][3], m[k][4], m[k][5])
    
    # Sherman-Morrison formula
    Hv   = np.array([[H[0]], [H[1]], [0], [0], [0]])
    HvT  = np.transpose(Hv)
    
    div = V + np.dot(np.dot(HvT, icov), Hv)[0, 0]
    res = np.dot(np.dot(np.dot(icov, Hv), HvT), icov)
    res = np.true_divide(res, div)
    
    fcov = np.subtract(icov, res)
    if abs(np.linalg.det(fcov)) < 1.e-30:
        print("error, final covariance matrix is singular!")
        return

    # filter the state vector
    for j in range(5):
        K[j] = (H[0] * fcov[j,0] + H[1] * fcov[j,1])/V
        
    h = geth(ivec[1], ivec[2], m[k][3], m[k][4], m[k][5])
    chi2 += (m[k][1] - h) * (m[k][1] - h) / V
    
    fvec = np.copy(ivec)
    for j in range(5):
        fvec[j+1] += K[j] * (m[k][1] - h)
    
    if prnt >= 2:
        print("  filter:     ", fvec)
    if prnt >= 3:
        print("    matrix f:\n", fcov)
        print("    chi2: %3f\n" % chi2)
        
    return (fvec, fcov, chi2)

### Kalman Filter

In [31]:
def kfitter(I, s, S, Z, B, prnt):
    K = len(Z) # total number of measurements
    v = 0.002997924580 # speed of light
    
    # best last state vector, covariance matrix and fit's chi^2 fit probability found
    sK_b = np.zeros(6)
    SK_b = np.zeros((5,5))
    b_chi2 = np.Inf
    
    # run the Kalman Filter
    for i in range(I):
        c_chi2 = 0
        if i > 0:
            (s[0], S[0]) = k_transport(K-1, 0, s[K-1], S[K-1], v, Z, B, prnt)
        for k in range(1, K):
            (s[k], S[k]) = k_transport(k-1, k, s[k-1], S[k-1], v, Z, B, prnt)
            (s[k], S[k], c_chi2) = k_filter(k, s[k], S[k], c_chi2, prnt)
        if np.isnan(c_chi2): # NOTE: IDK why this is happening
            break
        if c_chi2 >= b_chi2:
            if prnt >= 1: print("iteration %2d : no improvement" % i)
            continue
        if prnt >= 1:
            print("iteration %2d :" % i, s[K-1], " (%.2f)" % c_chi2)
        if  abs(s[K-1][5] - sK_b[5]) < 5.e-4 and \
            abs(s[K-1][1] - sK_b[1]) < 1.e-4 and \
            abs(s[K-1][2] - sK_b[2]) < 1.e-4 and \
            abs(s[K-1][3] - sK_b[3]) < 1.e-6 and \
            abs(s[K-1][4] - sK_b[4]) < 1.e-6:
            i = I
        sK_b = s[K-1]
        SK_b = S[K-1]
        b_chi2 = c_chi2

    if i == 1:
        sK_b = s[len(s)-1]
        SK_b = S[len(s)-1]
    
    return (sK_b, SK_b)

### Get data from files

In [32]:
(Z, m) = read_meas_file(MEASFILENAME)
B = read_magfield_file(MAGFIELDFILENAMES[1][0])

### KFitter

In [33]:
# config
prnt = 1 # how much info to print (from 0 to 3)
I = 30 # number of iterations

# state vector and covariance matrix arrays
s = []
S = []

for i in range(len(Z)):
    s.append(np.zeros(6))
    S.append(np.zeros((5,5)))

# initial state vector and covariance matrix
s[0] = np.array([Z[0], 12.834542, 18.629339,  0.048048,  0.102449, -0.588641])
S[0] = np.array(
    [[27.809679,  0.941768, -0.109315, -0.004673,  0.009938],
     [ 0.941768, 85.895073, -0.002793, -0.410291,  0.003917],
     [-0.109315, -0.002793,  0.000481,  0.000020, -0.000130],
     [-0.004673, -0.410291,  0.000020,  0.002811, -0.000046],
     [ 0.009938,  0.003917, -0.000130, -0.000046,  0.000733]]
)

(sK_b, SK_b) = kfitter(I, s, S, Z, B, prnt)


[-50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -40.0, -40.0, -40.0, -40.0, -40.0, -40.0, -40.0, -40.0, -40.0, -40.0, -40.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -30.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -20.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, -10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
[-50.00000 -40.00000 -30.00000 -20.00000 -10.00000  0.00000  10.00000
  20.00000  30.00000  40.00000  50.00000]
12.834542
18.629339
[-50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -40.0, -40.0, -40.0

ValueError: The requested sample points xi have dimension 1, but this RegularGridInterpolator has dimension 2